<a href="https://colab.research.google.com/github/gauravreddy08/learning-pytorch/blob/main/07_pytorch_experiment_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 | **PyTorch** Experiment Tracking
https://www.learnpytorch.io/07_pytorch_experiment_tracking/

In [ ]:
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 796.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
import torchvision
import wandb

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: \"{device}\"")

Device: "cuda"


In [ ]:
run = wandb.init(project="learning-pytorch")

test_acc,▁██▇▆▇▇█▅▅
test_loss,█▆▅▃▃▄▂▁▂▂
train_acc,▁▄▆▅▆▆▆▆▇█
train_loss,█▆▅▄▃▃▂▂▂▁
test_acc,0.81534
test_loss,0.51089
train_acc,0.9375
train_loss,0.45743


In [ ]:
def set_seed(seed: int = 42):
  torch.manual_seed(42)
  torch.cuda.manual_seed(42)

In [ ]:
import os
import zipfile

from pathlib import Path
import requests

def download_data(source: str,
                  destination: str,
                  remove_source: bool = True) -> Path:
    data = Path("data/")
    data_path = data /destination
    if(data_path.is_dir()):
      print("[INFO] Data already exists...")
    else:
      print(f"[INFO] Did not find {data_path} directory, creating one...")
      data_path.mkdir(parents=True, exist_ok=True)

      target_file = Path(source).name
      with open(data/target_file, 'wb') as f:
        request = requests.get(source)
        print(f"[INFO] Downloading {target_file} from {source}...")
        f.write(request.content)
      with zipfile.ZipFile(data/target_file, 'r') as zip_ref:
        print(f"[INFO] Unzipping {target_file} data...")
        zip_ref.extractall(data_path)
      if remove_source:
        os.remove(data/target_file)

    return data_path

data_path = download_data("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip", 
                          destination='pizaa_steak_sushi')
data_path

[INFO] Did not find data/pizaa_steak_sushi directory, creating one...
[INFO] Downloading pizza_steak_sushi.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip...
[INFO] Unzipping pizza_steak_sushi.zip data...


PosixPath('data/pizaa_steak_sushi')

In [ ]:
train_dir = data_path/'train'
test_dir = data_path/'test'

In [ ]:
!git clone https://github.com/gauravreddy08/learning-pytorch
!mv /content/learning-pytorch/going_modular .
!rm -rf learning-pytorch

Cloning into 'learning-pytorch'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 56 (delta 21), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (56/56), 1.76 MiB | 7.79 MiB/s, done.


In [ ]:
from going_modular import engine, data_setup

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT 
auto_transforms = weights.transforms()

train_dataloader, test_dataloader, class_names = data_setup.create_dataset(train_dir=train_dir,
                                                                           test_dir=test_dir,
                                                                           transform=auto_transforms) 

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7f3d0f714550>,
 ['pizza', 'steak', 'sushi'])

In [ ]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

In [ ]:
try:
  import torchinfo
except:
  !pip -q install torchinfo
  import torchinfo

In [ ]:
for param in model.features.parameters():
  param.requires_grad = False

set_seed()

model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, 
              out_features=len(class_names),
              bias=True).to(device))

In [ ]:
from torchinfo import summary

summary(model, input_size=(32, 3, 224, 224),
        col_names=["input_size", "output_size", "trainable"])

Layer (type:depth-idx)                                  Input Shape               Output Shape              Trainable
EfficientNet                                            [32, 3, 224, 224]         [32, 3]                   Partial
├─Sequential: 1-1                                       [32, 3, 224, 224]         [32, 1280, 7, 7]          False
│    └─Conv2dNormActivation: 2-1                        [32, 3, 224, 224]         [32, 32, 112, 112]        False
│    │    └─Conv2d: 3-1                                 [32, 3, 224, 224]         [32, 32, 112, 112]        False
│    │    └─BatchNorm2d: 3-2                            [32, 32, 112, 112]        [32, 32, 112, 112]        False
│    │    └─SiLU: 3-3                                   [32, 32, 112, 112]        [32, 32, 112, 112]        --
│    └─Sequential: 2-2                                  [32, 32, 112, 112]        [32, 16, 112, 112]        False
│    │    └─MBConv: 3-4                                 [32, 32, 112, 112]        [32

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=0.001)

In [ ]:
wandb.watch(model, log_freq=1, idx=1)  
results = engine.train(model, epochs=5, 
                       train_data=train_dataloader, test_data=test_dataloader,
                       loss=loss_fn, optimizer=optimizer)

EPOCH [0]
Train Loss: 1.0467, Train Acc: 0.39% | Test Loss: 0.8372, Test Acc: 0.73% 
------------
EPOCH [1]
Train Loss: 0.9123, Train Acc: 0.63% | Test Loss: 0.7120, Test Acc: 0.89% 
------------
EPOCH [2]
Train Loss: 0.7654, Train Acc: 0.74% | Test Loss: 0.6891, Test Acc: 0.89% 
------------
EPOCH [3]
Train Loss: 0.7187, Train Acc: 0.74% | Test Loss: 0.5999, Test Acc: 0.87% 
------------
EPOCH [4]
Train Loss: 0.6184, Train Acc: 0.78% | Test Loss: 0.5966, Test Acc: 0.85% 
------------


In [ ]:
wandb.watch(model, log_freq=1, idx=2)  
results = engine.train(model, epochs=5, 
                       train_data=train_dataloader, test_data=test_dataloader,
                       loss=loss_fn, optimizer=optimizer)

EPOCH [0]
Train Loss: 0.5873, Train Acc: 0.80% | Test Loss: 0.6131, Test Acc: 0.86% 
------------
EPOCH [1]
Train Loss: 0.5425, Train Acc: 0.80% | Test Loss: 0.5123, Test Acc: 0.87% 
------------
EPOCH [2]
Train Loss: 0.5128, Train Acc: 0.79% | Test Loss: 0.4789, Test Acc: 0.89% 
------------
EPOCH [3]
Train Loss: 0.5205, Train Acc: 0.82% | Test Loss: 0.5139, Test Acc: 0.83% 
------------
EPOCH [4]
Train Loss: 0.4574, Train Acc: 0.94% | Test Loss: 0.5109, Test Acc: 0.82% 
------------


In [ ]:
from datetime import datetime

name = "model_" + datetime.now().strftime("%d%M%y_%H%M%S")+ '.pth'
torch.save(obj=model.state_dict(), f=name)
print(f"[INFO] Saved model as {name}")

[INFO] Saved model as model_094323_074308.pth


In [ ]:
artifact = wandb.Artifact("FoodVisionMini", type="model")
artifact.add_file('/content/model_094323_074308.pth')

run.log_artifact(artifact)

In [ ]:
artifact = wandb.Artifact("FoodVisionMiniData", type="dataset")
artifact.add_dir('/content/data/pizaa_steak_sushi')

run.log_artifact(artifact)

wandb: Adding directory to artifact (/content/data/pizaa_steak_sushi)... Done. 0.2s
